In [1]:
from amino_acids import aa, codons, aa_table
import random
import doctest

In [2]:
# A dictionary for returning the complements
completment_list = {'A':'T','C':'G', 'G':'C', 'T':'A'}
def get_complement(nucleotide):
    return completment_list[nucleotide.upper()]

In [3]:
get_complement('C')

'G'

In [4]:
def get_reverse_complement(dna):
    """ Computes the reverse complementary sequence of DNA for the specfied DNA
        sequence

        dna: a DNA sequence represented as a string
        returns: the reverse complementary DNA sequence represented as a string
    >>> get_reverse_complement("ATGCCCGCTTT")
    'AAAGCGGGCAT'
    >>> get_reverse_complement("CCGCGTTCA")
    'TGAACGCGG'
    """
    # Create a empty string to hold the reverse complement
    reverse_complement = ''
    
    # Add the nucleotide complements to the reverse complement string
    for nucleotide in dna:
        reverse_complement += get_complement(nucleotide)
    return reverse_complement[::-1]

doctest.testmod(verbose = False)

TestResults(failed=0, attempted=2)

In [5]:
start_codon = codons[3][0]
stop_codon = codons[10]

def rest_of_ORF(dna):
    """ Takes a DNA sequence that is assumed to begin with a start
        codon and returns the sequence up to but not including the
        first in frame stop codon.  If there is no in frame stop codon,
        returns the whole string.

        dna: a DNA sequence
        returns: the open reading frame represented as a string
    >>> rest_of_ORF("ATGTGAA")
    'ATG'
    >>> rest_of_ORF("ATGAGATAGG")
    'ATGAGA'
    """
    i = 0
    ORF = ''
    
    # As long as the current position of the curser is not beyond the dna length,
    # read three as a group until hit a stop codon
    while i < len(dna):
        #print(dna)
        # Save the current i as the index to start recording 3 necleuctides
        old_i = i
        # Add 3 to i to get the end index
        i += 3
        # store the 3 necleuctides as one in a temporary variable
        temp = dna[old_i:i:1]
        
        # check if the codon is a stop codon, return the ORF as is; otherwise
        # keep adding codons to the ORF
        if temp == stop_codon[0] or temp == stop_codon[1] or temp == stop_codon[2]:
            return ORF
        else:
            ORF += temp
    return ORF

doctest.testmod(verbose=False)

TestResults(failed=0, attempted=4)

In [6]:
def find_all_ORFs_oneframe(dna):
    """ Finds all non-nested open reading frames in the given DNA
        sequence and returns them as a list.  This function should
        only find ORFs that are in the default frame of the sequence
        (i.e. they start on indices that are multiples of 3).
        By non-nested we mean that if an ORF occurs entirely within
        another ORF, it should not be included in the returned list of ORFs.

        dna: a DNA sequence
        returns: a list of non-nested ORFs
    >>> find_all_ORFs_oneframe("ATGCATGAATGTAGATAGATGTGCCC")
    ['ATGCATGAATGTAGA', 'ATGTGCCC']
    """
    checked = False
    ORF_list = []
    # base_frame is the base from which to look for a starting codon
    base_frame = dna
    # base_frame_num is the starting index for a start codon
    base_frame_num = base_frame.find(start_codon)
    
    # Making sure that the frames stay in sync by making sure the ATG location
    # can be divided by three
    #print(base_frame_num)
    
    if base_frame_num % 3 != 0:
        while 1:
            #print(i)
            
            base_frame_num += 3
            temp_frame = base_frame[base_frame_num:]
#             print(temp_frame)
            temp_num = temp_frame.find(start_codon)
#             print(temp_num)
            if temp_num == -1:
                    return
            base_frame_num += temp_num
            #print(base_frame_num)
            if base_frame_num % 3 == 0:
                base_frame = base_frame[base_frame_num:]
                #print(base_frame)
                checked = True
                break
            

        
    # Keep looking for an ORF until when there's no more to read or when it
    # would be a frame out of the original sync
    while 1:
        # base_frame_num would be negative when there is no starting codon to
        # be found, in this case, the loop should break and allow the function
        # to return the ORF collected up to this point
        if base_frame_num < 0:
            break
#         Get one full frame from rest_of_ORF() and append it to the ORF_list
#         print(base_frame[base_frame_num:], ' Hello')
        if checked:
            full_frame = rest_of_ORF(base_frame)
        else:
            full_frame = rest_of_ORF(base_frame[base_frame_num:])
#         print(full_frame)
#         print(full_frame)
        ORF_list.append(full_frame)
        # Look for the next ATG index from a base_frame with the previously
        # found full_frame removed from the start
        next_ATG_loc = base_frame[len(full_frame):].find(start_codon)
        # if the index of the next ATG is not divisible by 3 (meaning that
        # the new start codon isn't in sync with the previuos one), then
        # break from the looking for another ORF
        if (next_ATG_loc % 3) != 0:
            break
        else:
            base_frame = base_frame[len(full_frame)+3:]
            #print(base_frame)
            base_frame_num = next_ATG_loc-3
    
    return ORF_list

doctest.testmod(verbose=False)

TestResults(failed=0, attempted=5)

In [7]:
def find_all_ORFs(dna):
    """ Finds all non-nested open reading frames in the given DNA sequence in
        all 3 possible frames and returns them as a list.  By non-nested we
        mean that if an ORF occurs entirely within another ORF and they are
        both in the same frame, it should not be included in the returned list
        of ORFs.

        dna: a DNA sequence
        returns: a list of non-nested ORFs

    >>> find_all_ORFs("ATGCATGAATGTAG")
    ['ATGCATGAATGTAG', 'ATGAATGTAG', 'ATG']
    """
    all_ORF = []
    # Loop through the possibilities of a starting codon being on the multiples
    # of 0, 1, 2 index
    for i in range(3):
#         print(dna[i:])
        temp_all_frame = find_all_ORFs_oneframe(dna[i:])
        # In case that there are more than just one frame in one find_all,
        # append individual ones to the list with the for loop
        if temp_all_frame != None:
            for y in range(len(temp_all_frame)):
                all_ORF.append(temp_all_frame[y])
            
    return all_ORF

doctest.testmod(verbose=False)

TestResults(failed=0, attempted=6)

In [8]:
def find_all_ORFs_both_strands(dna):
    """ Finds all non-nested open reading frames in the given DNA sequence on both
        strands.

        dna: a DNA sequence
        returns: a list of non-nested ORFs
    >>> find_all_ORFs_both_strands("ATGCGAATGTAGCATCAAA")
    ['ATGCGAATG', 'ATGCTACATTCGCAT']
    """
    ORF_both = []
    dna_complement = get_reverse_complement(dna)
    original_ORF = find_all_ORFs(dna)
    complement_ORF = find_all_ORFs(dna_complement)
    for i in range(len(original_ORF)):
        ORF_both.append(original_ORF[i])
    for i in range(len(complement_ORF)):
        ORF_both.append(complement_ORF[i])
    return ORF_both

doctest.testmod(verbose = False)

TestResults(failed=0, attempted=7)

In [9]:
squares = [x**2 for x in range(10)]
squares

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

# Week Two Stuff

In [10]:
def longest_ORF(dna):
    """ Finds the longest ORF on both strands of the specified DNA and returns it
        as a string
    >>> longest_ORF("ATGCGAATGTAGCATCAAA")
    'ATGCTACATTCGCAT'
    """
    ORF_both = find_all_ORFs_both_strands(dna)
    longest_length = 0
    longest_length_index = None
    for i in range(len(ORF_both)):
        if len(ORF_both[i]) > longest_length:
            longest_length_index = i
            longest_length = len(ORF_both[i])
    if longest_length_index == None:
        return '0'
    else:
        return ORF_both[longest_length_index]

In [11]:
longest_ORF("ATGCGAATGTAGCATCAAA")

'ATGCTACATTCGCAT'

In [12]:
import multiprocessing as mp
import random
import string

def longest_ORF_noncoding(dna, num_trials):
    """ Computes the maximum length of the longest ORF over num_trials shuffles
        of the specfied DNA sequence

        dna: a DNA sequence
        num_trials: the number of random shuffles
        returns: the maximum length longest ORF """
    
    random.seed()

    # Define an output queue
    output = mp.Queue()

    def rand_dna(dna, output):
        """ Generates a random string of numbers, lower- and uppercase chars. """
        length = len(dna)
        rand_str = ''.join(random.sample(dna, length))
        rand_str = longest_ORF(rand_str)
        output.put(rand_str)

    # Setup a list of processes that we want to run
    shuffle_process = [mp.Process(target=rand_dna, args=(dna, output)) for num in range(num_trials)]

    # Run processes
    for p in shuffle_process:
        p.start()

    # Exit the completed processes
    for p in shuffle_process:
        p.join()

    # Get process results from the output queue
    results = [output.get() for p in shuffle_process]
    
    return max(results)


In [18]:
%time longest_ORF_noncoding("AAGTCCAAATAGTGAGATCAAGTCCAAATAGTGAGATCAAGTCCAAATAGTGAGATC", 800)

CPU times: user 325 ms, sys: 1.16 s, total: 1.49 s
Wall time: 1.41 s


'ATGTTTTTTACTAGCTCTCAATTTG'

In [14]:
import multiprocessing as mp
import random
import string

random.seed()

# Define an output queue
output = mp.Queue()

# define a example function
def rand_string(length, output):
    """ Generates a random string of numbers, lower- and uppercase chars. """
    rand_str = ''.join(random.choice(
                        string.ascii_lowercase
                        + string.ascii_uppercase
                        + string.digits)
                   for i in range(length))
    output.put(rand_str)

# Setup a list of processes that we want to run
processes = [mp.Process(target=rand_string, args=(5, output)) for x in range(6)]

# Run processes
for p in processes:
    p.start()

# Exit the completed processes
for p in processes:
    p.join()

# Get process results from the output queue
results = [output.get() for p in processes]

print(results)

['LVnVZ', 'GGhYf', 'x7Dkz', 'FKDOx', 'H1AR2', 'pf2Xy']


In [15]:
random.seed()

# Define an output queue
output = mp.Queue()

dna_dna = "ATGCGAATGTAGCATCAAA"

def rand_dna(dna, output):
    """ Generates a random string of numbers, lower- and uppercase chars. """
    length = len(dna)
    rand_str = ''.join(random.sample(dna, length))
    rand_str = longest_ORF(rand_str)
    output.put(rand_str)

# Setup a list of processes that we want to run
processes = [mp.Process(target=rand_dna, args=(dna_dna, output)) for x in range(5)]

# Run processes
for p in processes:
    p.start()

# Exit the completed processes
for p in processes:
    p.join()

# Get process results from the output queue
results = [output.get() for p in processes]

print(results)

['ATGACTTCGAAGAA', 'ATGTCGAAAAAC', '0', 'ATGCAAGTTCAAGTA', 'ATG']


In [16]:
rand_dna(dna_dna, output)
print(output.get())

ATGCATGAAATCAAAGC
